# Examples

In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.datasets import load_boston
from dynopt.models.models import Model

## Example 1. Linear Regression with Boston Housing Data

In [2]:
from sklearn.datasets import load_boston

# Load dataset
data = load_boston()

# Define data names
feature_names = data.feature_names.tolist()
target_name = 'MEDV'  # Median value of homes in $1000s

# Construct Pandas dataframes
X = pd.DataFrame(data.data, columns=feature_names)
y = pd.DataFrame(data.target, columns=[target_name])

print(X.head())

      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  
0     15.3  396.90   4.98  
1     17.8  396.90   9.14  
2     17.8  392.83   4.03  
3     18.7  394.63   2.94  
4     18.7  396.90   5.33  


In [3]:
print(y.head())

   MEDV
0  24.0
1  21.6
2  34.7
3  33.4
4  36.2


In [4]:
# Select desired inputs and outputs
x_names = ['LSTAT', 'RM', 'TAX']
y_names = ['MEDV']

# Initialize model
model = Model(x_names, y_names)  # by default uses a linear estimator

print(model)

Model(['LSTAT', 'RM', 'TAX'], ['MEDV'], estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False))


In [5]:
# Fit model
model.fit(X, y)

# Fit score (R-squared)
print(model.score(X, y))

0.6485147669915273


In [6]:
# Prediction with fitted model

print(model.predict(X.head()))

        MEDV
0  29.012111
1  26.263785
2  33.059827
3  32.819835
4  32.273937


In [7]:
# Single point prediction is also supported
x = {'LSTAT': 4.98, 'RM': 6.575, 'TAX': 296}
print(model.predict(x))

{'MEDV': 29.01211141973685}


## Simple Pendulum Example

In [8]:
import numpy as np

def pendulum_dydt(t, y):
    dydt = np.empty_like(y)
    dydt[0] = y[1]
    dydt[1] = -y[1] - 5*np.sin(y[0])
    return dydt

In [9]:
from scipy.integrate import odeint

# Integrate dy/dt over time
t = np.arange(0, 10, 0.1)
y0 = [0.1, 0]  # Initial condition
y = odeint(pendulum_dydt, y0, t, tfirst=True)
assert y.shape == (100, 2), y.shape

In [10]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6,2.5))
for i in range(y.shape[1]):
    plt.plot(t, y[:,i], label=f'y[{i}]')
plt.xlabel('t')
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig('images/pendulum-time-plot.png')
plt.show()

<Figure size 600x250 with 1 Axes>

In [11]:
# Calculate dydt values
dydt = pendulum_dydt(t, y.T).T
assert dydt.shape == (100, 2)

In [12]:
from dynopt.models.models import NonLinearModel

# Labels for inputs and outputs
x_names = ['theta', 'theta_dot']
y_names = ['theta_dot', 'theta_ddot']

# Choose input features including non-linear terms
x_features = ['x1', 'sin(x0)']

# Initialize model
model = NonLinearModel(x_names, y_names, x_features=x_features)

In [13]:
# Prepare pandas dataframes for inputs and outputs
y = pd.DataFrame(y, columns=x_names)
dydt = pd.DataFrame(dydt, columns=y_names)
print(y.head())

      theta  theta_dot
0  0.100000   0.000000
1  0.097595  -0.047109
2  0.090802  -0.087513
3  0.080359  -0.119943
4  0.067106  -0.143621


In [14]:
print(dydt.head())

   theta_dot  theta_ddot
0   0.000000   -0.499167
1  -0.047109   -0.440093
2  -0.087513   -0.365874
3  -0.119943   -0.281420
4  -0.143621   -0.191657


In [15]:
# Fit the linear model
model.fit(y, dydt)
print(model.coef_.round(5))

     x1  sin(x0)
y0  1.0     -0.0
y1 -1.0     -5.0


In [16]:
print(model.intercept_.round(5))

y0   -0.0
y1    0.0
dtype: float64


In [17]:
from dynopt.models.models import SparseNonLinearModel

# Labels for inputs and outputs
x_names = ['theta', 'theta_dot']
y_names = ['theta_dot', 'theta_ddot']

# Choose additional non-linear features for model identification
custom_features = ['sin(x0)', 'cos(x0)', 'sin(x0)**2', 
                   'cos(x0)**2', 'sin(x0)*cos(x0)']

# Initialize SINDy model
model = SparseNonLinearModel(x_names, y_names, 
                             custom_features=custom_features,
                             poly_order=2)

In [18]:
# Prepare pandas dataframes for inputs and outputs
y = pd.DataFrame(y, columns=x_names)
dydt = pd.DataFrame(dydt, columns=y_names)

In [19]:
# Sparse identification of non-linear dynamics (SINDy)
threshold = 0.2  # Sparsity parameter
model.fit(y, dydt, threshold=threshold)
print(model.n_params)

3


In [20]:
print(model.coef_)

     x1  sin(x0)
y0  1.0      0.0
y1 -1.0     -5.0
